In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install pickle5

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from glob import glob
import pickle5 as pickle
import numpy as np

ROOT_PATH = "../input/"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]

def get_city_trajectories(city="palo-alto", split="train", normalized=False, preprocess=False):
    if split=="train":
        f_in = ROOT_PATH + split + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        inputs = np.asarray(inputs).astype(np.float64)

        outputs = None
    
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs).astype(np.float64)
        
    elif split=="test":
        f_in = ROOT_PATH + "test-inputs" + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        inputs = np.asarray(inputs).astype(np.float64)

        outputs = None
    
    if normalized:
        train_combined = np.concatenate([inputs, outputs], axis=1)
        x_mean = np.mean(train_combined[:, :, 0].reshape(-1))
        x_std = np.std(train_combined[:, :, 0].reshape(-1))
        x_norm = ((train_combined[:, :, 0].reshape(-1) - x_mean) / x_std).reshape(train_combined.shape[0], train_combined.shape[1], 1)

        y_mean = np.mean(train_combined[:, :, 1].reshape(-1))
        y_std = np.std(train_combined[:, :, 1].reshape(-1))
        y_norm = ((train_combined[:, :, 1].reshape(-1) - y_mean) / y_std).reshape(train_combined.shape[0], train_combined.shape[1], 1)

        normalized = np.concatenate([x_norm, y_norm], axis=2)
        
        return normalized[:, :50, :], normalized[:, 50:, :]

    if preprocess:
        preprocessed_inp = []
        preprocessed_out = []
        for i in range(inputs.shape[0]):
            temp_i, temp_o = preprocess_single(inputs[i], outputs[i])
            preprocessed_inp.append(temp_i)
            preprocessed_out.append(temp_o)

        preprocessed_inp = np.array(preprocessed_inp)
        preprocessed_out = np.array(preprocessed_out)
        
        return preprocessed_inp, preprocessed_out
    
    return inputs, outputs

class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, city: str, split:str, transform=None, normalize=False, preprocess=False):
        super(ArgoverseDataset, self).__init__()
        self.transform = transform

        self.inputs, self.outputs = get_city_trajectories(city=city, split=split, normalized=normalize, preprocess=preprocess)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):

        data = (self.inputs[idx], self.outputs[idx])
            
        if self.transform:
            data = self.transform(data)

        return data

# intialize a dataset
city = 'palo-alto' 
split = 'train'
train_dataset  = ArgoverseDataset(city = city, split = split)

In [5]:
# here inp and out are single-target
def preprocess_single(inp, out):
    x_i = inp[0][0]
    y_i = inp[0][1]
    
    prep_inp = np.concatenate([(inp[:, 0] - x_i).reshape(50, 1), (inp[:, 1] - y_i).reshape(50, 1)], axis=1)
    
    if out is None:
        return prep_inp, None

    prep_out = np.concatenate([(out[:, 0] - x_i).reshape(60, 1), (out[:, 1] - y_i).reshape(60, 1)], axis=1)
    
    return prep_inp, prep_out

In [6]:
import torch.nn as nn

In [7]:
class Pred(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(100, 64),
            nn.ReLU(),
#             nn.Linear(64, 32),
            nn.Linear(64, 16)
#             nn.ReLU(),
#             nn.Linear(32, 16),
#             nn.ReLU(),
#             nn.Linear(16, 16)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(16, 120),
            nn.ReLU(),
#             nn.Linear(32, 64),
#             nn.ReLU(),
#             nn.Linear(64, 128),
#             nn.ReLU(),
#             nn.Linear(128, 120),
#             nn.ReLU(),
            nn.Linear(120, 120)
        )
        
    def forward(self, x):
        x = x.reshape(-1, 100).float()
        x = self.encoder(x)
        x = self.decoder(x)
#         print("------")
#         print(x.shape)
        x = x.reshape(-1, 60, 2)
        return x

In [8]:
pred = Pred()
opt = torch.optim.Adam(pred.parameters(), lr=0.0001)

In [9]:
batch_sz = 512  # batch size 
city = 'miami' 
split = 'train'
train_dataset  = ArgoverseDataset(city = city, split = split, normalize=False, preprocess=True)
train_loader = DataLoader(train_dataset, batch_size=batch_sz)

In [ ]:
# p_i, p_o = get_city_trajectories(city="palo-alto", split="train", normalized=False)
# train_combined = np.concatenate([p_i, p_o], axis=1)
# global_x_mean = np.mean(train_combined[:, :, 0].reshape(-1))
# global_x_std = np.std(train_combined[:, :, 0].reshape(-1))
# global_y_mean = np.mean(train_combined[:, :, 1].reshape(-1))
# global_y_std = np.std(train_combined[:, :, 1].reshape(-1))

In [ ]:
# global_x_mean

In [10]:
def train(pred):
    loss_lst = []
    for epoch in range(5000):
        total_loss = 0
        i = 0
        for i_batch, sample_batch in enumerate(train_loader):
            inp, out = sample_batch

#             print(inp.shape)
#             print(inp)
            preds = pred(inp)
#             print(preds.shape)
#             print(out.shape)
            loss = ((preds - out) ** 2).sum()

            opt.zero_grad()
            loss.backward()
            opt.step()
            
            i += len(inp) * 60 * 2
            
            total_loss += loss.item()
            
        loss_lst.append(total_loss / i)
        if epoch % 100 == 0:
            print('epoch {} loss: {}'.format(epoch, total_loss / i))
            
    plt.plot(loss_lst)
    
    return pred

In [11]:
import time

In [12]:
start = time.time()
print("Starts training process")
model = train(pred)
end = time.time()
print("Ends training process, total time - " + str(end - start))

In [13]:
model

In [14]:
p_test_i, p_test_o = get_city_trajectories(city="miami", split="test")

In [15]:
p_test_i.shape

In [ ]:
# def normalize(p_test_i):
#     return np.concatenate([((p_test_i[:, 0] - global_x_mean)/global_x_std).reshape(1,50,1), \
#                 ((p_test_i[:, 1] - global_y_mean)/global_y_std).reshape(1,50,1)], axis=2)

In [ ]:
model(torch.tensor(normalize(p_test_i[0])))

In [ ]:
start = time.time()
print("Starts training process")
model_1 = train(model)
end = time.time()
print("Ends training process, total time - " + str(end - start))

In [ ]:
p_test_i, p_test_o = get_city_trajectories(city="palo-alto", split="test", normalized=False)

In [ ]:
model_1

In [ ]:
start = time.time()
print("Starts training process")
model_2 = train(model_1)
end = time.time()
print("Ends training process, total time - " + str(end - start))

In [ ]:
start = time.time()
print("Starts training process")
model_3 = train(model_2)
end = time.time()
print("Ends training process, total time - " + str(end - start))

In [ ]:
model_3

In [16]:
def predict(model, test_inp):
    x_i = test_inp[0][0]
    y_i = test_inp[0][1]
    
    pred = model(torch.tensor(preprocess_single(test_inp, None)[0]))[0]
    
    pred_x = pred[:, 0] + x_i
    pred_y = pred[:, 1] + y_i
    
    return pred_x, pred_y

In [17]:
# here inp and out are single-target
def preprocess_single(inp, out):
    x_i = inp[0][0]
    y_i = inp[0][1]
    
    prep_inp = np.concatenate([(inp[:, 0] - x_i).reshape(50, 1), (inp[:, 1] - y_i).reshape(50, 1)], axis=1)
    
    if out is None:
        return prep_inp, None

    prep_out = np.concatenate([(out[:, 0] - x_i).reshape(60, 1), (out[:, 1] - y_i).reshape(60, 1)], axis=1)
    
    return prep_inp, prep_out

In [ ]:
model(torch.tensor(preprocess_single(p_test_i[0], p_test_o)[0])).shape

In [18]:
palo_df = pd.DataFrame()
for i in range(len(p_test_i)):
    
#     predict = model_3(torch.tensor(normalize(p_test_i[i])))
    p_x, p_y = predict(model, p_test_i[i])
    
    if i % 100 == 0:
        print("predicts for " + str(i) + " is done")
    temp_dict = {}
    for j in range(0, 120, 2):
        col_x = 'v' + str(j)
        col_y = 'v' + str(j+1)
        row = j // 2
#         temp_dict[col_x] = predict[0][row][0].item() * global_x_std + global_x_mean
#         temp_dict[col_y] = predict[0][row][1].item() * global_y_std + global_y_mean
        temp_dict[col_x] = p_x[row].item()
        temp_dict[col_y] = p_y[row].item()
    
    temp_idx_name = str(i) + "_" + "miami"
    temp_final = pd.DataFrame(temp_dict, index=[temp_idx_name])
    palo_df = pd.concat([palo_df, temp_final])

In [19]:
palo_df.reset_index(level=0).rename(columns={"index": "ID"}).to_csv("model5_miami.csv", index=False)